In [3]:
import sparseml
from sparseml.pytorch.optim import ScheduledModifierManager
from sparseml.pytorch.utils import get_prunable_layers, tensor_sparsity
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from tqdm.auto import tqdm
import math
from datasets import load_dataset

In [4]:
dataset = load_dataset("beans")
print(dataset)

Using custom data configuration default
Reusing dataset beans (/home/ubuntu/.cache/huggingface/datasets/beans/default/0.0.0/d5abfbb94de45599d871182cff389bc870bf145b4829c4b0fe20f0cccd637cbd)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 1034
    })
    validation: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 133
    })
    test: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 128
    })
})


In [6]:
print(dataset["train"][0]["image_file_path"])
print(dataset["validation"][0]["image_file_path"])

/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/eeb026374cf5ecfd5f40131a3159be9b9055ac21a3da11690e7eb4d117c99eee/train/bean_rust/bean_rust_train.84.jpg
/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/f287261265d2f9a3e8f87a5526a54d1847b17f7c3ec5714e5719432f2b3e4a73/validation/bean_rust/bean_rust_val.36.jpg


In [7]:
train_path = "/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/eeb026374cf5ecfd5f40131a3159be9b9055ac21a3da11690e7eb4d117c99eee/train"
val_path = "/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/f287261265d2f9a3e8f87a5526a54d1847b17f7c3ec5714e5719432f2b3e4a73/validation/" 

In [8]:
BATCH_SIZE = 32
NUM_LABELS = 3

# transforms
imagenet_transform = transforms.Compose([
   transforms.Resize(size=256, interpolation=transforms.InterpolationMode.BILINEAR, max_size=None, antialias=None),
   transforms.CenterCrop(size=(224, 224)),
   transforms.ToTensor(),
   transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# datasets
train_dataset = torchvision.datasets.ImageFolder(
    root=train_path,
    transform=imagenet_transform
)

val_dataset = torchvision.datasets.ImageFolder(
    root=val_path,
    transform=imagenet_transform
)

# setup data loaders
train_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=16)
val_loader = DataLoader(val_dataset, BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=16)

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
def run_model_one_epoch(model, data_loader, criterion, device, train=False, optimizer=None):
    if train:
        model.train()
    else:
        model.eval()

    running_loss = 0.0
    total_correct = 0
    total_predictions = 0

    # loop through batches
    for step, (inputs, labels) in tqdm(enumerate(data_loader), total=len(data_loader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        if train:
            optimizer.zero_grad()

        # compute loss, run backpropogation
        outputs = model(inputs)  # model returns logits
        loss = criterion(outputs, labels)
        if train:
            loss.backward()
            optimizer.step()

        running_loss += loss.item()

        # run evaluation
        predictions = outputs.argmax(dim=1)
        total_correct += torch.sum(predictions == labels).item()
        total_predictions += inputs.size(0)

    # return loss and evaluation metric
    loss = running_loss / (step + 1.0)
    accuracy = total_correct / total_predictions
    return loss, accuracy

## Train Dense Model

In [12]:
model = torchvision.models.mobilenet_v2(weights=torchvision.models.MobileNet_V2_Weights.DEFAULT)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, NUM_LABELS)
model.to(device)
print(model)
print(f"Using device: {device}")

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [18]:
# setup loss function and optimizer, LR will be overriden by sparseml
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=8e-3)

In [19]:
!cat ./dense_model/dense-recipe.yaml


# Epoch and Learning-Rate variables
num_epochs: 10.0
init_lr: 0.0005

training_modifiers:
  - !EpochRangeModifier
    start_epoch: 0.0
    end_epoch: eval(num_epochs)

  - !LearningRateFunctionModifier
    final_lr: 0.0
    init_lr: eval(init_lr)
    lr_func: cosine
    start_epoch: 0.0
    end_epoch: eval(num_epochs)

# Phase 1 Sparse Transfer Learning / Recovery
sparse_transfer_learning_modifiers:
  - !ConstantPruningModifier
    start_epoch: 0.0
    params: __ALL_PRUNABLE__


In [20]:
# create ScheduledModifierManager and Optimizer wrapper
manager = ScheduledModifierManager.from_yaml("./dense_model/dense-recipe.yaml")
optimizer = manager.modify(model, optimizer, steps_per_epoch=len(train_loader))

In [21]:
# run transfer learning
epoch = 0
for epoch in range(manager.max_epochs):
    # run training loop
    epoch_name = f"{epoch + 1}/{manager.max_epochs}"
    print(f"Running Training Epoch {epoch_name}")
    train_loss, train_acc = run_model_one_epoch(
        model, train_loader, criterion, device, train=True, optimizer=optimizer
    )
    print(
        f"Training Epoch: {epoch_name}\nTraining Loss: {train_loss}\nTop 1 Acc: {train_acc}\n"
    )

    # run validation loop
    print(f"Running Validation Epoch {epoch_name}")
    val_loss, val_acc = run_model_one_epoch(model, val_loader, criterion, device)
    print(
        f"Validation Epoch: {epoch_name}\nVal Loss: {val_loss}\nTop 1 Acc: {val_acc}\n"
    )

manager.finalize(model)

Running Training Epoch 1/10


  0%|          | 0/33 [00:00<?, ?it/s]

Training Epoch: 1/10
Training Loss: 0.49688769707625563
Top 1 Acc: 0.811411992263056

Running Validation Epoch 1/10


  0%|          | 0/5 [00:00<?, ?it/s]

Validation Epoch: 1/10
Val Loss: 0.10483174873515963
Top 1 Acc: 0.9624060150375939

Running Training Epoch 2/10


...

Validation Epoch: 9/10
Val Loss: 0.050181417539715764
Top 1 Acc: 0.9624060150375939

Running Training Epoch 10/10


  0%|          | 0/33 [00:00<?, ?it/s]

Training Epoch: 10/10
Training Loss: 0.006127264015517678
Top 1 Acc: 0.9980657640232108

Running Validation Epoch 10/10


  0%|          | 0/5 [00:00<?, ?it/s]

Validation Epoch: 10/10
Val Loss: 0.04676330639049411
Top 1 Acc: 0.9624060150375939



In [22]:
# print sparsities of each layer
for (name, layer) in get_prunable_layers(model):
    print(f"{name}.weight: {tensor_sparsity(layer.weight).item():.4f}")

features.0.0.weight: 0.0000
features.1.conv.0.0.weight: 0.0000
features.1.conv.1.weight: 0.0000
features.2.conv.0.0.weight: 0.0000
features.2.conv.1.0.weight: 0.0000
features.2.conv.2.weight: 0.0000
features.3.conv.0.0.weight: 0.0000
features.3.conv.1.0.weight: 0.0000
features.3.conv.2.weight: 0.0000
features.4.conv.0.0.weight: 0.0000
features.4.conv.1.0.weight: 0.0000
features.4.conv.2.weight: 0.0000
features.5.conv.0.0.weight: 0.0000
features.5.conv.1.0.weight: 0.0000
features.5.conv.2.weight: 0.0000
features.6.conv.0.0.weight: 0.0000
features.6.conv.1.0.weight: 0.0000
features.6.conv.2.weight: 0.0000
features.7.conv.0.0.weight: 0.0000
features.7.conv.1.0.weight: 0.0000
features.7.conv.2.weight: 0.0000
features.8.conv.0.0.weight: 0.0000
features.8.conv.1.0.weight: 0.0000
features.8.conv.2.weight: 0.0000
features.9.conv.0.0.weight: 0.0000
features.9.conv.1.0.weight: 0.0000
features.9.conv.2.weight: 0.0000
features.10.conv.0.0.weight: 0.0000
features.10.conv.1.0.weight: 0.0000
features

In [25]:
from sparseml.pytorch.utils import ModuleExporter

save_dir = "dense_model"
exporter = ModuleExporter(model, output_dir=save_dir)
exporter.export_pytorch(name="mobilenet-v2-dense-beans.pth")

## Prune The Model

In [27]:
checkpoint = torch.load("./dense_model/training/mobilenet-v2-dense-beans.pth")
model = torchvision.models.mobilenet_v2()
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, NUM_LABELS)
model.load_state_dict(checkpoint['state_dict'])
model.to(device)

# setup loss function and optimizer, LR will be overriden by sparseml
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=8e-3)

In [28]:
print(model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [29]:
!cat recipe-0.yaml

# Epoch and Learning-Rate variables
num_epochs: 13.0
pruning_epochs: 10.0
init_lr: 0.0005
inter_func: cubic
mask_type: unstructured

training_modifiers:
  - !EpochRangeModifier
    start_epoch: 0.0
    end_epoch: eval(num_epochs)

  - !LearningRateFunctionModifier
    final_lr: 0.0
    init_lr: eval(init_lr)
    lr_func: cosine
    start_epoch: 0.0
    end_epoch: eval(num_epochs)

# Pruning
pruning_modifiers:
  - !GlobalMagnitudePruningModifier
    init_sparsity: 0.05
    final_sparsity: 0.90
    start_epoch: 0.0
    end_epoch: eval(pruning_epochs)
    update_frequency: 1.0
    params: 
        - 'features.0.0.weight'
        - 'features.18.0.weight'
        - 're:features.*.conv.*.weight'
        - 're:features.*.conv.*.*.weight'
    leave_enabled: True
    inter_func: eval(inter_func)
    mask_type: eval(mask_type)

finetuning_modifiers:
  - !ConstantPruningModifier
    start_epoch: eval(pruning_epochs)
    params: __ALL_PRUNABLE__

In [30]:
# create ScheduledModifierManager and Optimizer wrapper
manager = ScheduledModifierManager.from_yaml("./recipe-0.yaml")
optimizer = manager.modify(model, optimizer, steps_per_epoch=len(train_loader))

In [31]:
for (name, layer) in get_prunable_layers(model):
    print(f"{name}.weight: {tensor_sparsity(layer.weight).item():.4f}")

features.0.0.weight: 0.0000
features.1.conv.0.0.weight: 0.0000
features.1.conv.1.weight: 0.0000
features.2.conv.0.0.weight: 0.0000
features.2.conv.1.0.weight: 0.0000
features.2.conv.2.weight: 0.0000
features.3.conv.0.0.weight: 0.0000
features.3.conv.1.0.weight: 0.0000
features.3.conv.2.weight: 0.0000
features.4.conv.0.0.weight: 0.0000
features.4.conv.1.0.weight: 0.0000
features.4.conv.2.weight: 0.0000
features.5.conv.0.0.weight: 0.0000
features.5.conv.1.0.weight: 0.0000
features.5.conv.2.weight: 0.0000
features.6.conv.0.0.weight: 0.0000
features.6.conv.1.0.weight: 0.0000
features.6.conv.2.weight: 0.0000
features.7.conv.0.0.weight: 0.0000
features.7.conv.1.0.weight: 0.0000
features.7.conv.2.weight: 0.0000
features.8.conv.0.0.weight: 0.0000
features.8.conv.1.0.weight: 0.0000
features.8.conv.2.weight: 0.0000
features.9.conv.0.0.weight: 0.0000
features.9.conv.1.0.weight: 0.0000
features.9.conv.2.weight: 0.0000
features.10.conv.0.0.weight: 0.0000
features.10.conv.1.0.weight: 0.0000
features

In [32]:
# run transfer learning
epoch = 0
for epoch in range(manager.max_epochs):
    # run training loop
    epoch_name = f"{epoch + 1}/{manager.max_epochs}"
    print(f"Running Training Epoch {epoch_name}")
    train_loss, train_acc = run_model_one_epoch(
        model, train_loader, criterion, device, train=True, optimizer=optimizer
    )
    print(
        f"Training Epoch: {epoch_name}\nTraining Loss: {train_loss}\nTop 1 Acc: {train_acc}\n"
    )

    # run validation loop
    print(f"Running Validation Epoch {epoch_name}")
    val_loss, val_acc = run_model_one_epoch(model, val_loader, criterion, device)
    print(
        f"Validation Epoch: {epoch_name}\nVal Loss: {val_loss}\nTop 1 Acc: {val_acc}\n"
    )
    
    print(f"\n\nChecking Sparsity Level:")
    for (name, layer) in get_prunable_layers(model):
        print(f"{name}.weight: {tensor_sparsity(layer.weight).item():.4f}")
    print(f"\n")
    
manager.finalize(model)

Running Training Epoch 1/13


  0%|          | 0/33 [00:00<?, ?it/s]

Training Epoch: 1/13
Training Loss: 0.05181791849088657
Top 1 Acc: 0.9835589941972921

Running Validation Epoch 1/13


  0%|          | 0/5 [00:00<?, ?it/s]

Validation Epoch: 1/13
Val Loss: 0.08574707275256514
Top 1 Acc: 0.9624060150375939



Checking Sparsity Level:
features.0.0.weight: 0.0880
features.1.conv.0.0.weight: 0.0729
features.1.conv.1.weight: 0.0820
features.2.conv.0.0.weight: 0.0293
features.2.conv.1.0.weight: 0.0081
features.2.conv.2.weight: 0.0169
features.3.conv.0.0.weight: 0.0321
features.3.conv.1.0.weight: 0.0255
features.3.conv.2.weight: 0.0286
features.4.conv.0.0.weight: 0.0240
features.4.conv.1.0.weight: 0.0008
features.4.conv.2.weight: 0.0197
features.5.conv.0.0.weight: 0.0343
features.5.conv.1.0.weight: 0.0237
features.5.conv.2.weight: 0.0288
features.6.conv.0.0.weight: 0.0319
features.6.conv.1.0.weight: 0.0272
features.6.conv.2.weight: 0.0334
features.7.conv.0.0.weight: 0.0218
features.7.conv.1.0.weight: 0.0035
features.7.conv.2.weight: 0.0228
features.8.conv.0.0.weight: 0.0445
features.8.conv.1.0.weight: 0.0341
features.8.conv.2.weight: 0.0502
features.9.conv.0.0.weight: 0.0424
features.9.conv.1.0.weight: 0.0292
fe

...

Validation Epoch: 12/13
Val Loss: 0.07313197031617165
Top 1 Acc: 0.9774436090225563



Checking Sparsity Level:
features.0.0.weight: 0.4919
features.1.conv.0.0.weight: 0.4618
features.1.conv.1.weight: 0.5156
features.2.conv.0.0.weight: 0.5495
features.2.conv.1.0.weight: 0.2951
features.2.conv.2.weight: 0.4852
features.3.conv.0.0.weight: 0.6152
features.3.conv.1.0.weight: 0.4915
features.3.conv.2.weight: 0.6137
features.4.conv.0.0.weight: 0.5185
features.4.conv.1.0.weight: 0.2670
features.4.conv.2.weight: 0.5167
features.5.conv.0.0.weight: 0.6847
features.5.conv.1.0.weight: 0.5880
features.5.conv.2.weight: 0.7153
features.6.conv.0.0.weight: 0.6929
features.6.conv.1.0.weight: 0.6169
features.6.conv.2.weight: 0.7378
features.7.conv.0.0.weight: 0.5461
features.7.conv.1.0.weight: 0.3646
features.7.conv.2.weight: 0.6134
features.8.conv.0.0.weight: 0.8370
features.8.conv.1.0.weight: 0.7410
features.8.conv.2.weight: 0.8693
features.9.conv.0.0.weight: 0.8263
features.9.conv.1.0.weight: 0.7367
f

  0%|          | 0/33 [00:00<?, ?it/s]

Training Epoch: 13/13
Training Loss: 0.08235379680991173
Top 1 Acc: 0.97678916827853

Running Validation Epoch 13/13


  0%|          | 0/5 [00:00<?, ?it/s]

Validation Epoch: 13/13
Val Loss: 0.07470075637102128
Top 1 Acc: 0.9774436090225563



Checking Sparsity Level:
features.0.0.weight: 0.4919
features.1.conv.0.0.weight: 0.4618
features.1.conv.1.weight: 0.5156
features.2.conv.0.0.weight: 0.5495
features.2.conv.1.0.weight: 0.2951
features.2.conv.2.weight: 0.4852
features.3.conv.0.0.weight: 0.6152
features.3.conv.1.0.weight: 0.4915
features.3.conv.2.weight: 0.6137
features.4.conv.0.0.weight: 0.5185
features.4.conv.1.0.weight: 0.2670
features.4.conv.2.weight: 0.5167
features.5.conv.0.0.weight: 0.6847
features.5.conv.1.0.weight: 0.5880
features.5.conv.2.weight: 0.7153
features.6.conv.0.0.weight: 0.6929
features.6.conv.1.0.weight: 0.6169
features.6.conv.2.weight: 0.7378
features.7.conv.0.0.weight: 0.5461
features.7.conv.1.0.weight: 0.3646
features.7.conv.2.weight: 0.6134
features.8.conv.0.0.weight: 0.8370
features.8.conv.1.0.weight: 0.7410
features.8.conv.2.weight: 0.8693
features.9.conv.0.0.weight: 0.8263
features.9.conv.1.0.weight: 0.7367
f

In [33]:
from sparseml.pytorch.utils import ModuleExporter

save_dir = "experiment-0"
exporter = ModuleExporter(model, output_dir=save_dir)
exporter.export_pytorch(name="mobilenet_v2-sparse-experiment-0.pth")
exporter.export_onnx(torch.randn(1, 3, 224, 224), name="sparse-model.onnx", convert_qat=True)

/home/ubuntu/rshaw/python-venvs/sparseml-env/lib/python3.8/site-packages/torch/onnx/utils.py:439: UserWarning: It is recommended that constant folding be turned off ('do_constant_folding=False') when exporting the model in training-amenable mode, i.e. with 'training=TrainingMode.TRAIN' or 'training=TrainingMode.PRESERVE' (when model is in training mode). Otherwise, some learnable model parameters may not translate correctly in the exported ONNX model because constant folding mutates model parameters. Please consider turning off constant folding or setting the training=TrainingMode.EVAL.
  warnings.warn(


In [34]:
checkpoint2 = torch.load("./dense_model/training/mobilenet-v2-dense-beans.pth")
model2 = torchvision.models.mobilenet_v2()
model2.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, NUM_LABELS)
model2.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [36]:
exporter = ModuleExporter(model2, output_dir="./test")
exporter.export_onnx(torch.randn(1, 3, 224, 224), name="dense-model.onnx", convert_qat=True)